# Fashion-MNIST PyTorch image classification w/ Tensorboard
Source
- https://tutorials.pytorch.kr/intermediate/tensorboard_tutorial.html
- https://github.com/aws/amazon-sagemaker-examples/blob/master/frameworks/pytorch/get_started_mnist_train.ipynb

## Initial setup

In [59]:
install_needed = True
# install_needed = False

In [60]:
import sys
import IPython

if install_needed:
    print("installing deps and restarting kernel")
    !{sys.executable} -m pip install -U 'sagemaker[local]'
    !{sys.executable} -m pip install -U sagemaker-experiments # SageMaker Experiments SDK 
    !{sys.executable} -m pip install -U sagemaker             # SageMaker Python SDK
    !/bin/bash ./local/local_mode_setup.sh
    IPython.Application.instance().kernel.do_shutdown(True)

installing deps and restarting kernel
     |████████████████████████████████| 466 kB 1.8 MB/s eta 0:00:01
     |████████████████████████████████| 131 kB 29.8 MB/s eta 0:00:01
     |████████████████████████████████| 8.4 MB 27.4 MB/s eta 0:00:01
  Created wheel for sagemaker: filename=sagemaker-2.70.0-py2.py3-none-any.whl size=649170 sha256=44c282dbe8d731cce57fa9b015346f3a1346d17b8afbcc1f3047fc2895037b1f
  Stored in directory: /home/ec2-user/.cache/pip/wheels/da/11/20/c45ef599886a2b1399effa68f80b98b2166dc624e19636c303
Successfully built sagemaker
  Attempting uninstall: botocore
    Found existing installation: botocore 1.23.4
    Uninstalling botocore-1.23.4:
      Successfully uninstalled botocore-1.23.4
  Attempting uninstall: boto3
    Found existing installation: boto3 1.20.4
    Uninstalling boto3-1.20.4:
      Successfully uninstalled boto3-1.20.4
  Attempting uninstall: sagemaker
    Found existing installation: sagemaker 2.68.0
    Uninstalling sagemaker-2.68.0:
      Successful

## Prepare dataset

In [29]:
# imports
import matplotlib.pyplot as plt
import numpy as np

import torch
import torchvision
import torchvision.transforms as transforms

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from time import strftime

In [2]:
# transforms
transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))])

# datasets
trainset = torchvision.datasets.FashionMNIST('./data',
    download=True,
    train=True,
    transform=transform)
testset = torchvision.datasets.FashionMNIST('./data',
    download=True,
    train=False,
    transform=transform)

## Set up the SageMaker environment

In [3]:
import os
import sagemaker
from sagemaker.pytorch import PyTorch
from sagemaker import get_execution_role

from smexperiments.experiment import Experiment ### SM Experiment
from smexperiments.trial import Trial           ### SM Experiment

from sagemaker.debugger import TensorBoardOutputConfig ### For TensorBoard 

sagemaker_session = sagemaker.Session()

role = get_execution_role()

bucket = sagemaker_session.default_bucket()
prefix = "tensorboard_pytorch_fashion_mnist"
tensorboard_logs_path = "s3://{}/{}/logs".format(bucket, prefix) ### For TensorBoard
output_path = "s3://{}/{}/output".format(bucket, prefix)

print("Bucket: {}".format(bucket))
print("SageMaker ver: " + sagemaker.__version__)
print("Tensorboard log path: {}".format(tensorboard_logs_path))

Bucket: sagemaker-ap-northeast-2-889750940888
SageMaker ver: 2.70.0
Tensorboard log path: s3://sagemaker-ap-northeast-2-889750940888/tensorboard_pytorch_fashion_mnist/logs


## Uploading the data to s3

In [4]:
!aws s3 cp ./data/FashionMNIST/raw s3://{bucket}/{prefix}/data --recursive

upload: data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to s3://sagemaker-ap-northeast-2-889750940888/tensorboard_pytorch_fashion_mnist/data/t10k-labels-idx1-ubyte.gz
upload: data/FashionMNIST/raw/t10k-labels-idx1-ubyte to s3://sagemaker-ap-northeast-2-889750940888/tensorboard_pytorch_fashion_mnist/data/t10k-labels-idx1-ubyte
upload: data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to s3://sagemaker-ap-northeast-2-889750940888/tensorboard_pytorch_fashion_mnist/data/train-labels-idx1-ubyte.gz
upload: data/FashionMNIST/raw/train-labels-idx1-ubyte to s3://sagemaker-ap-northeast-2-889750940888/tensorboard_pytorch_fashion_mnist/data/train-labels-idx1-ubyte
upload: data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to s3://sagemaker-ap-northeast-2-889750940888/tensorboard_pytorch_fashion_mnist/data/t10k-images-idx3-ubyte.gz
upload: data/FashionMNIST/raw/t10k-images-idx3-ubyte to s3://sagemaker-ap-northeast-2-889750940888/tensorboard_pytorch_fashion_mnist/data/t10k-images-idx3-ubyte
upload: data

In [5]:
train_location = 's3://{}/{}/data'.format(bucket, prefix)
test_location = 's3://{}/{}/data'.format(bucket, prefix)

In [6]:
!aws s3 ls {train_location} --recursive

2021-12-09 13:42:32    7840016 tensorboard_pytorch_fashion_mnist/data/t10k-images-idx3-ubyte
2021-12-09 13:42:32    4422102 tensorboard_pytorch_fashion_mnist/data/t10k-images-idx3-ubyte.gz
2021-12-09 13:42:32      10008 tensorboard_pytorch_fashion_mnist/data/t10k-labels-idx1-ubyte
2021-12-09 13:42:32       5148 tensorboard_pytorch_fashion_mnist/data/t10k-labels-idx1-ubyte.gz
2021-12-09 13:42:32   47040016 tensorboard_pytorch_fashion_mnist/data/train-images-idx3-ubyte
2021-12-09 13:42:32   26421880 tensorboard_pytorch_fashion_mnist/data/train-images-idx3-ubyte.gz
2021-12-09 13:42:32      60008 tensorboard_pytorch_fashion_mnist/data/train-labels-idx1-ubyte
2021-12-09 13:42:32      29515 tensorboard_pytorch_fashion_mnist/data/train-labels-idx1-ubyte.gz


In [7]:
!aws s3 ls {test_location} --recursive

2021-12-09 13:42:32    7840016 tensorboard_pytorch_fashion_mnist/data/t10k-images-idx3-ubyte
2021-12-09 13:42:32    4422102 tensorboard_pytorch_fashion_mnist/data/t10k-images-idx3-ubyte.gz
2021-12-09 13:42:32      10008 tensorboard_pytorch_fashion_mnist/data/t10k-labels-idx1-ubyte
2021-12-09 13:42:32       5148 tensorboard_pytorch_fashion_mnist/data/t10k-labels-idx1-ubyte.gz
2021-12-09 13:42:32   47040016 tensorboard_pytorch_fashion_mnist/data/train-images-idx3-ubyte
2021-12-09 13:42:32   26421880 tensorboard_pytorch_fashion_mnist/data/train-images-idx3-ubyte.gz
2021-12-09 13:42:32      60008 tensorboard_pytorch_fashion_mnist/data/train-labels-idx1-ubyte
2021-12-09 13:42:32      29515 tensorboard_pytorch_fashion_mnist/data/train-labels-idx1-ubyte.gz


## Local mode training

In [8]:
# from sagemaker.local import LocalSession
# sagemaker_session = LocalSession()

In [9]:
from sagemaker.debugger import TensorBoardOutputConfig ### For TensorBoard 

# An error occurred (ValidationException) when calling the CreateTrainingJob operation:
# "LocalPath" of "TensorBoardOutputConfig" cannot start with the following reserved path: [/opt/ml, /tmp, /usr/local/nvidia]

tensorboard_output_config = TensorBoardOutputConfig(
    s3_output_path=tensorboard_logs_path,
    container_local_output_path='/pytorch/tensors'
) 

In [13]:
hyperparameters_local = {"batch-size": 128,
                         "epochs": 1,
                         "learning-rate": 1e-3,
                         "log-interval": 100,
                         "tensorboard-logs-path": tensorboard_logs_path} # Not working in local mode

In [15]:
# set local_mode to be True if you want to run the training script
# on the machine that runs this notebook

local_mode = True

if local_mode:
    instance_type = "local"
else:
    instance_type = "ml.c5.xlarge"

est_local = PyTorch(
            entry_point="train.py",
            source_dir="code",  # directory of your training script
            role=role,
            framework_version="1.8.1",
            py_version="py3",
            instance_type=instance_type,
            instance_count=1,
            output_path=output_path,
            hyperparameters=hyperparameters_local,
            tensorboard_output_config=tensorboard_output_config
)

In [16]:
channels = {"training": train_location, "testing": test_location}
est_local.fit(inputs=channels)

INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: latest.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker:Creating training-job with name: pytorch-training-2021-12-09-13-43-21-069
INFO:sagemaker.local.local_session:Starting training job
INFO:sagemaker.local.image:No AWS credentials found in session but credentials from EC2 Metadata Service are available.
INFO:sagemaker.local.image:docker compose file: 
networks:
  sagemaker-local:
    name: sagemaker-local
services:
  algo-1-suxwy:
    command: train
    container_name: l54b0oup5i-algo-1-suxwy
    environment:
    - '[Masked]'
    - '[Masked]'
    image: 763104351884.dkr.ecr.ap-northeast-2.amazonaws.com/pytorch-training:1.8.1-cpu-py3
    networks:
      sagemaker-local:
        aliases:
        - algo-1-suxwy
    stdin_open: true
    tty: true
    volumes:
    - /tmp/tmp7108yq45/algo-1-suxwy/output/data:/opt/ml/output/data
    - /tmp/tmp7108yq45/algo-1-suxwy/input

Creating l54b0oup5i-algo-1-suxwy ... 
Creating l54b0oup5i-algo-1-suxwy ... done
Attaching to l54b0oup5i-algo-1-suxwy
l54b0oup5i-algo-1-suxwy | 2021-12-09 13:43:25,660 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
l54b0oup5i-algo-1-suxwy | 2021-12-09 13:43:25,662 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
l54b0oup5i-algo-1-suxwy | 2021-12-09 13:43:25,670 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
l54b0oup5i-algo-1-suxwy | 2021-12-09 13:43:25,673 sagemaker_pytorch_container.training INFO     Invoking user training script.
l54b0oup5i-algo-1-suxwy | 2021-12-09 13:43:25,821 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
l54b0oup5i-algo-1-suxwy | /opt/conda/bin/python3.6 -m pip install -r requirements.txt
l54b0oup5i-algo-1-suxwy | Collecting tensorboard<2.4
l54b0oup5i-algo-1-suxwy |   Downloading tensorboard-2.3.0-py3-none-any.

## Managed training

### Experiments

In [17]:
def create_experiment(experiment_name):
    try:
        sm_experiment = Experiment.load(experiment_name)
    except:
        sm_experiment = Experiment.create(experiment_name=experiment_name,
                                          tags=[
                                              {
                                                  'Key': 'modelname',
                                                  'Value': 'fashion-mnist'
                                              },
                                          ])

In [18]:
def create_trial(experiment_name, i_type, i_cnt, spot):
    create_date = strftime("%m%d-%H%M%s")
    
    algo = 'dp'
    
    spot = 's' if spot else 'd'
    i_tag = 'test'
    
    if i_type == 'ml.p3.16xlarge':
        i_tag = 'p3'
    elif i_type == 'ml.p2.8xlarge':
        i_tag = 'p2'
    elif i_type == 'ml.p3dn.24xlarge':
        i_tag = 'p3dn'
    elif i_type == 'ml.p4d.24xlarge':
        i_tag = 'p4d'
    else:
        i_tag = 'others'
        
    trial = "-".join([i_tag,str(i_cnt),algo, spot])
       
    sm_trial = Trial.create(trial_name=f'{experiment_name}-{trial}-{create_date}',
                            experiment_name=experiment_name)

    job_name = f'{sm_trial.trial_name}'
    return job_name

### Debugger rules

In [19]:
from sagemaker.debugger import Rule, ProfilerRule, rule_configs

rules = [
    Rule.sagemaker(rule_configs.loss_not_decreasing()),
    ProfilerRule.sagemaker(rule_configs.LowGPUUtilization()),
    ProfilerRule.sagemaker(rule_configs.ProfilerReport()),
]

### Debugger Profiling

In [20]:
from sagemaker.debugger import ProfilerConfig, FrameworkProfile

profiler_config = ProfilerConfig(
    system_monitor_interval_millis=500, framework_profile_params=FrameworkProfile(num_steps=10)
)

### Training environments

In [21]:
metric_definitions = [{'Name': 'average loss',
                       'Regex': 'Average loss: ([0-9\\.]+)'},
                      {'Name': 'accuracy',
                       'Regex': 'Accuracy: [0-9]+/[0-9]+, ([0-9\\.]+)'}]

In [22]:
hyperparameters = {"batch-size": 128,
                   "epochs": 10,
                   "learning-rate": 1e-3,
                   "log-interval": 100,
                   "tensorboard-logs-path": tensorboard_logs_path}

In [26]:
# set local_mode to be True if you want to run the training script
# on the machine that runs this notebook

local_mode = False

instance_count = 1

if local_mode:
    instance_type = "local"
else:
    instance_type = "ml.p3.2xlarge"

estimator = PyTorch(
            entry_point="train.py",
            source_dir="code",  # directory of your training script
            role=role,
            framework_version="1.8.1",
            py_version="py3",
            instance_type=instance_type,
            instance_count=instance_count,
            output_path=output_path,
            hyperparameters=hyperparameters,
            tensorboard_output_config=tensorboard_output_config,
            base_job_name='pytorch-tensorboard',
            metric_definitions=metric_definitions,
            profiler_config=profiler_config,
            rules=rules,
            disable_profiler=False # default: False
)

In [30]:
experiment_name = 'pytorch-tensorboard'
do_spot_training=False

create_experiment(experiment_name)
job_name = create_trial(experiment_name, instance_type, instance_count, do_spot_training)
job_name

'pytorch-tensorboard-others-1-dp-d-1209-13451639057554'

In [32]:
channels = {"training": train_location, "testing": test_location}
estimator.fit(inputs=channels,
              experiment_config={
                  'TrialName': job_name,
                  'TrialComponentDisplayName': job_name,
                },
              wait=False)

INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: latest.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: latest.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: latest.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker:Creating training-job with name: pytorch-tensorboard-2021-12-09-13-46-49-120


In [34]:
job_name=estimator.latest_training_job.name
sagemaker_session.logs_for_job(job_name=job_name, wait=True)

2021-12-09 13:46:51 Starting - Launching requested ML instances...LossNotDecreasing: InProgress
LowGPUUtilization: InProgress
ProfilerReport: InProgress
......
2021-12-09 13:48:33 Starting - Preparing the instances for training.........
2021-12-09 13:49:55 Downloading - Downloading input data
2021-12-09 13:49:55 Training - Downloading the training image.......................bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2021-12-09 13:53:46,950 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2021-12-09 13:53:46,973 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2021-12-09 13:53:53,209 sagemaker_pytorch_container.training INFO     Invoking user training script.
2021-12-09 13:53:53,784 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
/opt/conda/bin/python3.6 -m pip install -r requirements.txt

2021-12-09 13

### Download profiler report

In [38]:
rule_output_path = estimator.output_path + '/' + estimator.latest_training_job.job_name + "/rule-output"
rule_output_path

's3://sagemaker-ap-northeast-2-889750940888/tensorboard_pytorch_fashion_mnist/output/pytorch-tensorboard-2021-12-09-13-46-49-120/rule-output'

In [39]:
! aws s3 ls {rule_output_path} --recursive

2021-12-09 13:55:22     389321 tensorboard_pytorch_fashion_mnist/output/pytorch-tensorboard-2021-12-09-13-46-49-120/rule-output/ProfilerReport/profiler-output/profiler-report.html
2021-12-09 13:55:21     239418 tensorboard_pytorch_fashion_mnist/output/pytorch-tensorboard-2021-12-09-13-46-49-120/rule-output/ProfilerReport/profiler-output/profiler-report.ipynb
2021-12-09 13:55:17        191 tensorboard_pytorch_fashion_mnist/output/pytorch-tensorboard-2021-12-09-13-46-49-120/rule-output/ProfilerReport/profiler-output/profiler-reports/BatchSize.json
2021-12-09 13:55:17      10527 tensorboard_pytorch_fashion_mnist/output/pytorch-tensorboard-2021-12-09-13-46-49-120/rule-output/ProfilerReport/profiler-output/profiler-reports/CPUBottleneck.json
2021-12-09 13:55:17       1927 tensorboard_pytorch_fashion_mnist/output/pytorch-tensorboard-2021-12-09-13-46-49-120/rule-output/ProfilerReport/profiler-output/profiler-reports/Dataloader.json
2021-12-09 13:55:17        129 tensorboard_pytorch_fashion_mn

In [41]:
os.makedirs('./profiler', exist_ok=True)

In [42]:
! aws s3 cp {rule_output_path} ./profiler --recursive

download: s3://sagemaker-ap-northeast-2-889750940888/tensorboard_pytorch_fashion_mnist/output/pytorch-tensorboard-2021-12-09-13-46-49-120/rule-output/ProfilerReport/profiler-output/profiler-reports/CPUBottleneck.json to profiler/ProfilerReport/profiler-output/profiler-reports/CPUBottleneck.json
download: s3://sagemaker-ap-northeast-2-889750940888/tensorboard_pytorch_fashion_mnist/output/pytorch-tensorboard-2021-12-09-13-46-49-120/rule-output/ProfilerReport/profiler-output/profiler-reports/Dataloader.json to profiler/ProfilerReport/profiler-output/profiler-reports/Dataloader.json
download: s3://sagemaker-ap-northeast-2-889750940888/tensorboard_pytorch_fashion_mnist/output/pytorch-tensorboard-2021-12-09-13-46-49-120/rule-output/ProfilerReport/profiler-output/profiler-reports/GPUMemoryIncrease.json to profiler/ProfilerReport/profiler-output/profiler-reports/GPUMemoryIncrease.json
download: s3://sagemaker-ap-northeast-2-889750940888/tensorboard_pytorch_fashion_mnist/output/pytorch-tensorbo